In [57]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
import google.generativeai as genai
import time
import requests
import ast
import json
from sklearn.metrics.pairwise import cosine_similarity
import re


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 2)

load_dotenv("../../.env",override=True)
GOOGLE_API_KEY = getenv("GEMINI_API_KEY")
OPENAI_API_KEY = getenv("OPENAI_API_KEY")


In [39]:

class chatGPT():
    def __init__(self):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        
    # def ask(self, q):
    #     stream = self.client.chat.completions.create(
    #         model="gpt-4",
    #         messages=[{"role": "user", "content": q}],
    #         stream=True,
    #         temperature=0
    #     )
    #     response = ""
    #     for chunk in stream:
    #         if chunk.choices[0].delta.content is not None:
    #             response += chunk.choices[0].delta.content

    #     self.response = response
    #     return response

    def get_embedding(self,text, model="text-embedding-3-large"):
       text = text.replace("\n", " ")
       return self.client.embeddings.create(input = [text], model=model).data[0].embedding

x_chat = chatGPT()


In [58]:
onet_df = pd.read_csv("../output/onet_df.csv")
onet_df = onet_df[(onet_df.task_similarity > .47) & (onet_df.job_title_similarity > .35)]
onet_weights = onet_df.groupby("organization_name")["onet_title"].count().apply(lambda x: 1/x).reset_index().rename({"onet_title":"onet_weight"},axis=1)
onet_df = onet_df.merge(onet_weights, on="organization_name")
onet_titles = onet_df.groupby("onet_title")["onet_weight"].sum().round(2).sort_values(ascending=False).reset_index()

onet_titles


,onet_title,onet_weight
0,Data Scientists,16.01
1,Search Marketing Strategists,15.74
2,Financial and Investment Analysts,10.30
3,Customer Service Representatives,9.50
4,Market Research Analysts and Marketing Special...,8.15
5,Investment Fund Managers,8.01
6,Graphic Designers,5.62
7,Writers and Authors,5.48
8,Travel Agents,5.43
9,Video Game Designers,5.27


In [33]:
onet_df.example_job_title

0            Legal Secretary
1            Legal Secretary
2            Legal Secretary
3            Legal Secretary
4            Legal Secretary
               ...          
229     vacation rental host
230     vacation rental host
231    vacation rental guest
232    vacation rental guest
233    vacation rental guest
Name: example_job_title, Length: 234, dtype: object

In [82]:
onet_df.onet_task.value_counts().reset_index().head()

,onet_task,count
0,Collect and analyze data on customer demograph...,20
1,"Forecast and track marketing and sales trends,...",18
2,Inform investment decisions by analyzing finan...,17
3,Apply feature selection algorithms to models p...,14
4,"Interpret data on price, yield, stability, fut...",14


In [ ]:
# sample_df = pd.read_csv("../output/df_with_onet.csv")
# sample_df = sample_df[~sample_df.Industry.isnull()]
# sample_df['Industry_embedding'] = sample_df['Industry'].apply(lambda text: x_chat.get_embedding(text))

In [75]:
industry_df = pd.read_csv("../output/df_with_industry_embeddings.csv")
embeddings = industry_df.Industry_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
embeddings = np.array(embeddings)
embeddings = np.vstack(embeddings)


In [92]:
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.neighbors import NearestNeighbors
# from sklearn.cluster import SpectralClustering
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

# n_neighbors = 7



# # Create a KNN model for connectivity based on cosine distance
# knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
# knn.fit(embeddings)
# connectivity_matrix = knn.kneighbors_graph(embeddings, mode='connectivity')

# # Use Spectral Clustering with the connectivity matrix defined by KNN
# clustering = SpectralClustering(n_clusters=200, affinity='precomputed', assign_labels='kmeans')
# cluster_labels = clustering.fit_predict(connectivity_matrix)

# # Assign cluster labels back to the original DataFrame
# sample_df['cluster_label'] = cluster_labels
# if "industry_cluster_label" in sample_df.columns:
#     sample_df = sample_df.drop(columns=['industry_cluster_label'])
# industry_clusters = pd.DataFrame(sample_df.groupby("cluster_label")['Industry'].agg(lambda x: ', '.join(x)).reset_index())


# industry_clusters = industry_clusters.rename({"Industry":"industry_cluster_label"},axis=1)
# sample_df = sample_df.merge(industry_clusters, on="cluster_label")


/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:248: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [105]:
sample_df['cluster_label'] = cluster_labels

if "industry_cluster_label" in sample_df.columns:
    sample_df = sample_df.drop(columns=['industry_cluster_label'])
industry_clusters = pd.DataFrame(sample_df.groupby("cluster_label")['Industry'].agg(lambda x: ', '.join(x)).reset_index())

industry_clusters = industry_clusters.rename({"Industry":"industry_cluster_label"},axis=1)
sample_df = sample_df.merge(industry_clusters, on="cluster_label")

industry_cluster_df = sample_df[["organization name","Industry","Industry_embedding","cluster_label","industry_cluster_label"]]
industry_cluster_df.industry_cluster_label.unique()


array(['Legal, Digital marketing, Agriculture',
       'Home Design, Game development',
       'Customer relationship management (CRM), Workflow Automation, Artificial intelligence, Mental health, Data Analytics, Web Development, Industrial maintenance',
       'Marketing , Climate technology,  AI software solutions',
       'Venture Capital, Hospitality, News aggregation, AI-powered research platform, Content creation, Public safety, Software Development, Knowledge management , Animal health, Marketing automation, Business automation, Fundraising, Financial services, Education technology',
       'IT Services, Financial Services, Business automation, Cybersecurity, AI-powered content creation , Legal Tech, Finance, Relationship Management, Website building, Video intelligence, Software development, IT Consulting, Retail, Restaurant management, Healthcare, Smart transportation, Legal Technology, Human Resources, AI-powered APIs, Talent Management , Healthcare,  Robotics, AI-powered cre

In [101]:
industry_df[industry_df.Industry.str.contains("Legal")]

,Unnamed: 0.1,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,headquarters region,website,actively hiring,linkedin,num articles,email,hub tag,phone num,num of sub org,industry groups,total funding amt,company type,estimated rev,description_all,industries_parsed,generated_description,Company,parsed_description,Tasks/Jobs,Industry,People Using Tool,examples,Job1_ONET,Job2_ONET,Job3_ONET,Example1,Job1,Example2,Job2,Example3,Job3,name,Job1_embedding,Job2_embedding,Job3_embedding,Industry_embedding,cluster_label
0,0,20738,Advoria,1-10,2023-08-29,Advoria's online appointment booking for law f...,"Artificial Intelligence (AI), Legal, Legal Tec...","Berlin, Berlin, Germany",Online appointment booking for the successful ...,"239,244",10967,"European Union (EU), Europe, Middle East, and ...",advoria.de,—,View on LinkedIn,—,mail@advoria.de,—,—,—,"Artificial Intelligence (AI), Data and Analyti...",—,For Profit,—,Advoria's online appointment booking for law f...,"['Legal', 'Legal Tech', 'SaaS', 'Software']","Advoria is a German-made, GDPR-compliant onlin...",Advoria,"\nTasks/Jobs: Appointment scheduling, Client i...","Appointment scheduling, Client intake, Secreta...",Legal,"Lawyers, Legal Secretaries",## Advoria Example Scenarios:\nExample 1: \nA ...,"[[""Legal Secretaries and Administrative Assist...","[[""Legal Secretaries and Administrative Assist...","[[""Legal Secretaries and Administrative Assist...",A lawyer uses Advoria to set up an automated b...,Legal Secretary schedules client appointments.,A law firm uses Advoria to collect necessary c...,Legal Secretary collects and organizes client ...,A paralegal uses Advoria to manage their own c...,Paralegal manages own calendar and appointment...,Advoria,"[-0.00684443861246109, -0.03683798760175705, -...","[-0.006861537229269743, -0.028195897117257118,...","[-0.011333281174302101, -0.03152986243367195, ...","[-0.002616604557260871, -0.00396003620699048, ...",136
70,70,22458,Fifty One Ai,1-10,2024-04-01,"At 51AI, a public benefit company, we're dedic...","Artificial Intelligence (AI), CivicTech, GovTe...","Arlington, Virginia, United States",AI with a Cause: Built to Scale Justice for Im...,—,22209,"Washington DC Metro Area, East Coast, Southern US",fiftyoneai.com,—,View on LinkedIn,1,wail@fiftyoneai.com,—,2028482282,—,"Artificial Intelligence (AI), Data and Analyti...",—,For Profit,—,"At 51AI, a public benefit company, we're dedic...","['CivicTech', 'GovTech', 'Legal Tech']",Fifty One AI is a public benefit company using...,Fifty One Ai,"\nTasks/Jobs: Report generation, Task automati...","Report generation, Task automation, Workflow o...",Legal,"Lawyers, Legal Aid Organizations",\nExample 1: A lawyer uses AI agents to automa...,NaN,NaN,NaN,A lawyer uses AI agents to automatically colle...,A legal assistant collects and organizes evide...,A legal aid organization uses AI to generate c...,A legal aid researcher conducts research and w...,A human rights organization deploys AI agents ...,A human rights researcher tracks and monitors ...,Fifty One Ai,"[-0.0030687362886965275, -0.006015349645167589...","[0.005640426650643349, -0.008820916526019573, ...","[-0.004961515311151743, -0.009353178553283215,...","[-0.0026129940524697304, -0.003935463260859251...",136
103,103,20871,SPEED AI,1-10,2023-09-01,The Legal Industry's First and ONLY AI-Powered...,"Legal, Legal Tech, Software",—,The Legal Industry's First and ONLY AI-Powered...,—,—,—,speedintake.com/,—,—,—,ai@wedrivecases.com,—,—,—,"Professional Services, Software",—,For Profit,—,The Legal Industry's First and ONLY AI-Powered...,"['Legal', 'Legal Tech', 'Software']",SPEED AI is a revolutionary AI-powered platfor...,SPEED AI,"\nTasks/Jobs: Intake process automation, Prosp...","Intake process automation, Prospect data analy...",Legal,"Lawyers, Legal professionals",\nExample 1: A lawyer uses SPEED AI to automat...,NaN,NaN,NaN,A lawyer uses SPEED AI to automate 

In [108]:
e = sample_df.Industry_embedding.iloc[0]
e

[-0.002616604557260871,
 -0.00396003620699048,
 -0.015124782919883728,
 0.021699294447898865,
 -0.016802474856376648,
 0.02806941792368889,
 -0.013114958070218563,
 0.05171040818095207,
 0.00712806498631835,
 0.04309200495481491,
 0.014639358967542648,
 0.006872578989714384,
 -0.05038188025355339,
 -0.0049308836460113525,
 0.012263337150216103,
 0.017151640728116035,
 -0.017543386667966843,
 -0.02694527804851532,
 -0.01588272489607334,
 0.030045177787542343,
 0.015669820830225945,
 0.029057297855615616,
 0.051608212292194366,
 0.016223372891545296,
 -0.011249909177422523,
 0.0030892540235072374,
 -0.019468048587441444,
 0.004577461164444685,
 0.0059954095631837845,
 -0.0013498187763616443,
 3.925439159502275e-06,
 0.0230789203196764,
 -0.010960358195006847,
 0.0006823610747233033,
 -0.035086773335933685,
 0.02083064243197441,
 0.02791612595319748,
 -0.0023589893244206905,
 -0.00580379506573081,
 -0.010091705247759819,
 0.001970437355339527,
 -0.02038779854774475,
 -0.024986550211906433

In [62]:
sample_df.to_csv("../output/df_with_industry_embeddings.csv")



